# Anopheles genomes phylogenetic scaffolding and evolution notebook

1. Yoann Aselmetti
2. Severine Berard
3. Eric Tannier
4. Cedric Chauve, Department of Mathematics, Simon Fraser University, cedric.chauve@sfu.ca

## Introduction

This notebook describes an improvement of the assembly of several mosquito genomes of the genus *Anopheles* using the newly developed phylogenetic scaffolding methods DeClone and ADseq.

In [1]:
import sys, math, numpy as np
%matplotlib inline  
import matplotlib, matplotlib.pyplot as plt

sys.path.insert(0, './scripts')
from declone_aux import *
from plotting import *

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Material and methods

In [2]:
# Reading the genes file
GENES_import(read_tab_file("./data/anopheles_genes_filtered"))
OG_import(read_tab_file("./data/anopheles_genes_filtered"))
GENOMES_import(read_tab_file("./data/anopheles_genes_filtered"))
# Reading the BESST file
BESST_import(read_tab_file("./data/anopheles_besst"))
# Reading the DeClone results file
DECLONE_import(read_tab_file("./results/anopheles_results_01_all"))

In [3]:
# Auxiliary functions
def __adj_score_species(a):
    return((DECLONE_adj_score(a),(DECLONE_adj_species_id(a),DECLONE_adj_species_name(a))))
def __adj_species(a):
    return((DECLONE_adj_species_id(a),DECLONE_adj_species_name(a)))
def __adj_genes(a):
    return(((DECLONE_adj_gene1_tree(a),DECLONE_adj_gene1_node(a)),(DECLONE_adj_gene2_tree(a),DECLONE_adj_gene2_node(a))))

In [ ]:
# Creating structures to access easily data and results
# Adjacencies: all adjacencies, ancestral adjacencies, extant adjacencies, scaffolding adjacencies
# Adjacencies can be interrogated using the DECLONE_adj_* functions from declone_aux.py
ALL_ADJ    = DECLONE_adjacencies_list()
ANC_ADJ    = [adj for adj in ALL_ADJ if DECLONE_adj_species_name(adj)=="ANCESTRAL"]
EXTANT_ADJ = [adj for adj in ALL_ADJ if DECLONE_adj_species_name(adj)!="ANCESTRAL"]
SCAFF_ADJ  = [adj for adj in EXTANT_ADJ if DECLONE_adj_score(adj)<1.0]
# Species: all species, extant species, ancestral species
# Each species is represented by a pair (node_id_in_newick_species_tree,name) where ancestral species have all the name "ANCESTRAL"
__ALL_SP_AUX={__adj_species(adj): 1 for adj in ALL_ADJ} # Temporary data structure
ALL_SPECIES    = [sp for sp in __ALL_SP_AUX.keys()]
EXTANT_SPECIES = [sp for sp in ALL_SPECIES if sp[1]!="ANCESTRAL"]
ANC_SPECIES    = [sp for sp in ALL_SPECIES if sp[1]=="ANCESTRAL"]

### Material

*Species tree (figure below), genes, chromosomes/scaffolds/contigs, gene families/orthogroups, gene trees*

<img src="images/anopheles_species_tree_labeled.png",width=600,height=600> 

In [ ]:
# Plotting the fragmentation and number of genes of each genome
GENOMES_EXTANT_FRAGMENTATION={sp[1]: {0: GENOMES_nbscf(sp[1])} for sp in EXTANT_SPECIES}
xlabel = "Number of scaffolds/contigs/chromosomes"
title  = "Assembly level of extant genomes"
plot_scores_distribution_per_species(GENOMES_EXTANT_FRAGMENTATION, 1, 0.5, xlabel, title,0.8,True)
GENOMES_EXTANT_NBGENES={sp[1]: {0: GENOMES_nbgenes(sp[1])} for sp in EXTANT_SPECIES}
xlabel = "Number of genes"
title  = "Genes complement of extant genomes"
plot_scores_distribution_per_species(GENOMES_EXTANT_NBGENES, 1, 0.5, xlabel, title,0.8,True)

### Methods

*ProfileNJ, DeClone, ADSeq, ideally joined into a single tool*

## Results

We now describe the results of our analysis, both in terms of scaffolding of extant *Anopheles* genomes, with a special focus on the genome *Anopheles funestus*, and of the ancestral genome maps we obtain, and what we can learn from them in terms of the evolution of the *Anopheles* genus.

In [ ]:
# Here we record the number of adjacencies for all species with adjacencies binned by bins of score 0.1
NB_BINS=10 # Binning the adjacencies by bins of size 0.1: 
           # bin i contains adjacencies of score <=0.1*i and <0.1*(i+1)
(IVALUES,IVALUES1)=(range(0,NB_BINS+1),range(0,NB_BINS))
ALL_ADJ_DISTRIB    = {sp: {t : 0 for t in IVALUES} for sp in ALL_SPECIES}
ANC_ADJ_DISTRIB    = {sp: {t : 0 for t in IVALUES} for sp in ANC_SPECIES}
EXTANT_ADJ_DISTRIB = {sp: {t : 0 for t in IVALUES} for sp in EXTANT_SPECIES}
SCAFF_ADJ_DISTRIB  = {sp: {t : 0 for t in IVALUES} for sp in EXTANT_SPECIES}
for adj in ALL_ADJ: 
    (score,species) = __adj_score_species(adj)
    score_bin=math.floor(score*NB_BINS)
    ALL_ADJ_DISTRIB[species][score_bin]+=1
    if species[1]=="ANCESTRAL":
        ANC_ADJ_DISTRIB[species][score_bin]+=1
    else:
        EXTANT_ADJ_DISTRIB[species][score_bin]+=1
        SCAFF_ADJ_DISTRIB[species][score_bin]+=1*int(score<1.0)

### Gene trees

*To do: describe the improvement of the new gene trees, gene content, duplications, ...*

### Scaffolding extant genomes: overview

In this section, we present a general overview of the scaffolding adjacencies inferred by the methods ADseq+DeClone.

In [ ]:
# Distribution of the scores of scaffolding adjacencies 
# CC: I do not know how to generate a nice table: this is a pis-aller
print("species name\t"+"\t".join(["<"+str(((t+1)/NB_BINS)) for t in IVALUES1]))
print("\n".join(species[1][0:15]+"\t"+"\t".join(str(SCAFF_ADJ_DISTRIB[species][t]) for t in IVALUES1) for species in EXTANT_SPECIES))
# Figure
xlabel= "Number of scaffolding adjacencies per DeClone score (bins of size "+str(1.0/NB_BINS)+")"
title = "Distribution of DeClone scores"
plot_scores_distribution_per_species(SCAFF_ADJ_DISTRIB, NB_BINS, 1.0, xlabel, title, 0.8,True)

We can observe on the table and figure above that many adjacencies seem to have a high DeClone score, although for highly fragmented genomes, we can observe a large number of poorly supported adjacencies. 

We now look at the number of conflicts, defined as either a gene with three or more neighbours, without accounting for gene orientation in extant species.


In [ ]:
# We now look at the conflicts
# We record the number of genes that belong to a conflict if restricted to all adjacencies of score 
# above a threshold. The thresholds are organized in bins of size 0.1
ANC_CONF_DISTRIB   = {sp: {t : 0 for t in IVALUES} for sp in ANC_SPECIES}
SCAFF_CONF_DISTRIB = {sp: {t : 0 for t in IVALUES} for sp in EXTANT_SPECIES}
DEGREE={} # Number of neighbours of genes, indexed by pairs (tree,node)

for adj in ALL_ADJ: # Initializing the tables
    (gene1,gene2)=__adj_genes(adj)
    (DEGREE[gene1],DEGREE[gene2])=({t : 0 for t in IVALUES},{t : 0 for t in IVALUES})
for adj in ALL_ADJ: # Filling the tables of gene neighbours
    (score,species)=__adj_score_species(adj)
    score_bin=math.floor(score*NB_BINS)
    (gene1,gene2)=__adj_genes(adj)
    for i in range (0,score_bin+1):
        DEGREE[gene1][i]+=1
        DEGREE[gene2][i]+=1
    __TAB_TO_UPDATE=ANC_CONF_DISTRIB
    if species[1]!="ANCESTRAL":
        __TAB_TO_UPDATE=SCAFF_CONF_DISTRIB
    if DEGREE[gene1][i]==3:
        for i in range (0,score_bin+1):
            __TAB_TO_UPDATE[species][i]+=1
    if DEGREE[gene2][i]==3:
        for i in range (0,score_bin+1):
            __TAB_TO_UPDATE[species][i]+=1

In [ ]:
# CC: I do not know how to generate a nice table: this is a pis-aller
print("species name\t"+"\t".join(["<"+str(((t+1)/NB_BINS)) for t in IVALUES1]))
print("\n".join(species[1][0:15]+"\t"+"\t".join(str(SCAFF_CONF_DISTRIB[species][t]) for t in IVALUES1) for species in EXTANT_SPECIES))
# Figure
xlabel= "Number of genes in conflicts in extant genomes"
title = "Syntenic conflicts in extant genomes"
plot_scores_distribution_per_species(SCAFF_CONF_DISTRIB, NB_BINS, 1.0, xlabel, title,0.8,True)

### *Anopheles funestus* scaffolding

### Ancestral genomes and evolution

The DeClone/ADseq method infers both extant and ancestral adjacencies, thus paving the way to analyse the evolution of genome organization over the considered species tree.

#### Overview of ancestral adjacencies



We first look at the distribution of scores of ancestral adjacencies.


In [ ]:
# CC: I do not know how to generate a nice table: this is a pis-aller
print("sp_id\t"+"\t".join(["<"+str(((t+1)/NB_BINS)) for t in IVALUES1])+"\t=1.0")
print("\n".join(species[0]+"\t"+"\t".join(str(ANC_ADJ_DISTRIB[species][t]) for t in IVALUES) for species in ANC_SPECIES))
# Figure
xlabel= "Number of ancestral adjacencies per DeClone score (bins of size "+str(1.0/NB_BINS)+")"
title = "Distribution of DeClone scores"
plot_scores_distribution_per_species(ANC_ADJ_DISTRIB, NB_BINS+1, 1.0, xlabel, title, 0.8,True)

We now look at the number of genes in a syntenic conflict, again considering adjacencies binned by DeClone score.

In [ ]:
# CC: I do not know how to generate a nice table: this is a pis-aller
print("sp_id\t"+"\t".join(["<"+str(((t+1)/NB_BINS)) for t in IVALUES1])+"\t=1.0")
print("\n".join(species[0]+"\t"+"\t".join(str(ANC_CONF_DISTRIB[species][t]) for t in IVALUES) for species in ANC_SPECIES))
# Figure
xlabel= "Number of adjacencies with at least one gene in conflicts in ancestral genomes"
title = "Syntenic conflicts in ancestral genomes"
plot_scores_distribution_per_species(ANC_CONF_DISTRIB, NB_BINS+1, 1.0, xlabel, title,0.8,True)

*To do: looking at the number of components without clearing conflicts, and then with clearing conflicts with the maximum weight matching algorithm.*

#### Compared evolution of rearrangement rates in sex chromosomes and autosomes

*Definition*: a reconciled gene tree is *X* if all its *A. gambia* genes are on the X chromosome, *autosome* if all its genes are on autosomes, *X+losses* if its genes are either losses or on the sex chromosome, *aut+losses* if the genes are losses or on autosomes, and finally *ambiguous* if it has genes on both the sex chromosome and aome autosome. 

As seen below, we can observe very few ambiguous reconciled gene trees, although a significant level of uncertainty is brought by gene losses.

However, when looking at instances, i.e. at pairs of gene families where at least one potential adjacency is obserrved in the extant genomes, we can see a higher level of apparent movement between the X chromosome and autosomes.

In [14]:
__GAMB_GT_FILE=open('results/gambia_trees','r').readlines()
__GT_CLASSES=["chrom.X","autosome", "X+losses", "aut+losses", "ambiguous"]
__GT_CLASSES_PRINT={"chrom.X": "chrom.X   ","autosome": "autosome", "X+losses": "X+losses", "aut+losses": "aut+losses", "ambiguous": "ambiguous"}
GT_CLASSES={gt.split("\t")[0]: gt.rstrip().split("\t")[1] for gt in __GAMB_GT_FILE if gt[0]!="#"}
GT_CLASSES_SUMMARY={c: len([GT_CLASSES[gt] for gt in GT_CLASSES.keys() if GT_CLASSES[gt]==c]) for c in __GT_CLASSES}
INSTANCES_CLASSES_SUMMARY={c1: {c2: 0 for c2 in __GT_CLASSES} for c1 in __GT_CLASSES}
for inst in DECLONE_instances_list():
    (tree1,tree2)=(DECLONE_instance_tree1(inst),DECLONE_instance_tree2(inst))
    INSTANCES_CLASSES_SUMMARY[GT_CLASSES[tree1]][GT_CLASSES[tree2]]+=1
for c in __GT_CLASSES:
    print(__GT_CLASSES_PRINT[c]+"\t"+str(GT_CLASSES_SUMMARY[c])+"\t"+str(INSTANCES_CLASSES_SUMMARY[c]))

chrom.X   	811	{'X+losses': 18, 'ambiguous': 22, 'autosome': 106, 'chrom.X': 1064, 'aut+losses': 252}
autosome	8168	{'X+losses': 0, 'ambiguous': 25, 'autosome': 10023, 'chrom.X': 105, 'aut+losses': 2930}
X+losses	23	{'X+losses': 7, 'ambiguous': 1, 'autosome': 8, 'chrom.X': 44, 'aut+losses': 10}
aut+losses	2511	{'X+losses': 0, 'ambiguous': 6, 'autosome': 1379, 'chrom.X': 47, 'aut+losses': 917}
ambiguous	21	{'X+losses': 1, 'ambiguous': 4, 'autosome': 13, 'chrom.X': 9, 'aut+losses': 8}


*To do: analyse the file results/adjacencies_evolution_chrom that contains the precise evolution of adjacencies along the branches of the species tree with the X/autosome status of the involved genes.*

## Discussion

## References